# Classes and Inheritance

- [Item 22: Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples](#Item-22:-Prefer-Helper-Classes-Over-Bookkeeping-with-Dictionaries-and-Tuples)
- [Item 23: Accept Functions for Simple Interfaces Instead of Classes](#Item-23:-Accept-Functions-for-Simple-Interfaces-Instead-of-Classes)

## Item 22: Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

In [ ]:
import collections

Grade = collections.namedtuple('Grade', ('score', 'weight'))

class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight
    

class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count
    

class Gradebook(object):
    def __init__(self):
        self._students = {}
    
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]
    
    
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grade(90, 0.1)
math.report_grade(100, 0.9)
print(albert.average_grade())

- Avoid making dictionaries with values that are other dictionaries or long tuples.
- Use `collections.namedtuple` for lightweight, immutable data containers before you need the flexibility of a full class.

## Item 23: Accept Functions for Simple Interfaces Instead of Classes

Many of Python's built-in APIs allow you to customize behavior by passing in a function. These hooks are used by APIs to call back your code while they execute

In [ ]:
names = ['Apollo', 'Muse', 'Zeus', 'Venus']
names.sort(key=lambda x: len(x))
print(names)

In [ ]:
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count  # Stateful closure
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
        
    return result, added_count


result, count = increment_with_report(current, increments)
assert count == 2


# class way
class CountMissing(object):
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0
    
counter = CountMissing()
result = defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount
assert counter.added == 2


class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0
    
    
counter = BetterCountMissing()
counter()
assert callable(counter)

result = defaultdict(counter, current)
for key, amount in increments:
    result[key] += amount
assert counter.added == 2

